# Anonymizing DICOM

In this exercise you will try your hand at anonymizing a DICOM series. It is very important to understand what is involved in anonymization and how it is done if you are planning to do any research projects with real clnical data. Either you will need to agree with the data owner on what the anonymization protocol is, or you will be dealing with data that passed such protocol. As always, understanding your data helps you build better machine learning algorithms.

You will find the series in a directory called `data`. By the way, that is an image of my own knee MRI that is coming directly from a scanner, with minimal editing. Among other things HIPAA guarantees patients access to their own data and lets them share the data as they see fit.  

For this exercise, imagine that you are starting a joint research project with a hospital and they have given you a sample image series to agree on anonymiaztion protocol for their data. You will need to come up with one before you see any further data.

In this notebook, I will guide you through a thought process for approaching such task, and have you perform the actual de-identification of DICOM metadata (note that there is also a lot that could be done with pixels, but we leave it outside the scope of this exercise). 

Keep in mind that anonymization process is always a balance between reducing chances of re-identification to a reasonable minimum and keeping the dataset useful for your research purposes.  

Another important disclaimer - while it is very useful to understand what exactly is involved in anonymization so that you can nail this balance perfectly, it is always a good idea to evaluate existing de-identification libraries for your task first.

Since we will be relying on PyDicom here, it's good to keep the reference handy: https://pydicom.github.io/pydicom/stable/tutorials/dataset_basics.html

In [1]:
import pydicom
import os

In [2]:
path = "data/KneeMr"
dcm = [pydicom.dcmread(os.path.join(path, f)) for f in os.listdir(path)]

In [3]:
len(dcm)

36

Great, we have series with 36 slices. Let's print metadata from one of those

In [4]:
img = dcm[0]
img

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND']
(0008, 0012) Instance Creation Date              DA: '20150116'
(0008, 0013) Instance Creation Time              TM: '160254.937000'
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.2.33.37105.2015011616025459632728241
(0008, 0020) Study Date                          DA: '20150116'
(0008, 0021) Series Date                         DA: '20150116'
(0008, 0022) Acquisition Date                    DA: '20150116'
(0008, 0023) Content Date                        DA: '20150116'
(0008, 0030) Study Time                          TM: '154708'
(0008, 0031) Series Time                         TM: '160254.859000'
(0008, 0032) Acquisition Time                    TM: '160008.295000'
(0008, 0033) Content Time                        TM: '160254.937000'
(0008, 0050)

## HIPAA Guidelines

Now, let's recall HIPAA guidelines for what constitutes PHI (also found [here](https://www.law.cornell.edu/cfr/text/45/164.514)):

>(A) Names;  
(B) All geographic subdivisions smaller than a State, including street address, city, county, precinct, zip code, and their equivalent geocodes, except for the initial three digits of a zip code if, according to the current publicly available data from the Bureau of the Census:  
    (1) The geographic unit formed by combining all zip codes with the same three initial digits contains more than 20,000 people; and  
    (2) The initial three digits of a zip code for all such geographic units containing 20,000 or fewer people is changed to 000.  
(C) All elements of dates (except year) for dates directly related to an individual, including birth date, admission date, discharge date, date of death; and all ages over 89 and all elements of dates (including year) indicative of such age, except that such ages and elements may be aggregated into a single category of age 90 or older;  
(D) Telephone numbers;  
(E) Fax numbers;  
(F) Electronic mail addresses;  
(G) Social security numbers;  
(H) Medical record numbers;  
(I) Health plan beneficiary numbers;  
(J) Account numbers;  
(K) Certificate/license numbers;  
(L) Vehicle identifiers and serial numbers, including license plate numbers;  
(M) Device identifiers and serial numbers;  
(N) Web Universal Resource Locators (URLs);  
(O) Internet Protocol (IP) address numbers;  
(P) Biometric identifiers, including finger and voice prints;  
(Q) Full face photographic images and any comparable images; and  
(R) Any other unique identifying number, characteristic, or code, except as permitted by paragraph (c) of this section; ...

Looks quite clear. Let's look at a specific data element. Let's say, Patient Name (0010,0010):

In [5]:
img.PatientName

'Tarapov, Ivan'

Ok, this seems clearly falling into category A) above. Let's clean it:

In [6]:
img.PatientName = "Anonymous"

Looks quite good, perhaps we should just continue doing this for everything suspicious we find? Let's look at another data element. Let's say, Station Name (0008,1010) - surely looks suspicious:

In [7]:
img[(0x0008,0x1010)]

(0008, 1010) Station Name                        SH: 'MRC37975'

Looks like something that might fall into category R) from above. But how do we know for certain? Also, should we put dummy data into this field? Should this data conform to some format? Remember all the DICOM data types?

I had mentioned the [online DICOM reference by Innolitics](https://dicom.innolitics.com/ciods) before - it would be of a great help in our case. Let's go over there and check what the standard has to say. We are dealing with MR IOD (Modality tag tells us this), so we are looking for this: https://dicom.innolitics.com/ciods/mr-image/general-equipment/00081010

The standard says the following:  
*Tag*: (0008,1010)  
*Type*: Optional (3)  
*Value Representation*: Short String (SH)  
*Description*: User defined name identifying the machine that produced the Composite Instances.  

Clearly, this looks like something the radiology department uses to identify their machines and we want to clean it to remove the PHI, as defined by HIPAA. 

Since it's an Optional tag, we can just get rid of it like so:

In [8]:
del img.StationName

Seems quite clear, right? Well, to be absolutely certain, let's take a look at another tag. In this case, let's look at Accession Number (0008,0050)

We are dealing with MR IOD, so we are looking for: https://dicom.innolitics.com/ciods/mr-image/general-study/00080050

The standard says the following:  
*Tag*: (0008,0050)  
*Type*: Required, Empty if Unknown (2)  
*Value Representation*: Short String (SH)  
*Description*: A RIS generated number that identifies the order for the Study.

So, it's a unique ID set by the RIS (remember Radiology Information System - the systems that stores administrative data for radiology department?) to uniquely identify the clinician's order for this study. In other words, a clinican who wants to look up this study would have this number in their medical record system and would try to use it to find the study.

Looks like we should keep it as it is required, but we can replace it with something. What can we replace it with? Remember the [value representations](http://dicom.nema.org/medical/dicom/2020a/output/chtml/part05/sect_6.2.html) from one of earlier lessons?  

SH, or Short String means 16 characters maximum, with some caveats. So it should be appropriate to replace it with something like "" or "DELETED", right?  

But what if you wanted to retain the ability to distinguish between studies with different accession numbers (e.g. you have other data from the patient visit and you want to be able to relate those)? In that case you could replace it with something like "MRKNEE1":

In [9]:
img.AccessionNumber = "MRKNEE1"

## DICOM Confidentiality Profiles

Seems like we need to make quite a lot of decisions about all these tags. There are also dates and UIDs. What do you do about these? Is there an easier way?  

Turns out there is. DICOM standard made anonymization easy for us by defining "*Attribute Confidentiality Profiles*". Basically, this is a list of all tags that could contain personal information and a set of guidelines for how to handle them. Attribute Confidentiality profiles are available here: http://dicom.nema.org/medical/dicom/2020a/output/chtml/part15/chapter_E.html

If you are building a real DICOM anonymizer, you should really study these guidelines well because they contain a lot of useful information that goes beyond of what we cover in this lesson. For the purpose of this exercise we will use only the most important bits.  

If you follow the link above you will find a set of possible operations (and corresponding codes) on tags such as:  

D - replace with a non-zero length value that may be a dummy value and consistent with the VR  
Z - replace with a zero length value, or a non-zero length value that may be a dummy value and consistent with the VR  
X - remove  
K - keep (unchanged for non-sequence attributes, cleaned for sequences)  
C - clean, that is replace with values of similar meaning known not to contain identifying information and consistent with the VR  
U - replace with a non-zero length UID that is internally consistent within a set of Instances  

You will also find a long table where rows contain individual tags and colums contain entries for "Basic Confidentiality Profile" and then some options. The way this works is that you decide which option suits your need by looking up what all the options mean on [this page of the standard](http://dicom.nema.org/medical/dicom/2020a/output/chtml/part15/sect_E.3.html), look up each tag and see what you are supposed to do with it, taking options into account. Options may be things like the intent to retain longitudinal information with full dates, or the intent to retain device identity. This is very convenient since this way the standard provides a common language to speak about identification. I.e. you may communicate in your release notes (or research protocol) that you perform DICOM anonymization per Basic Profile with "Clean Descriptiors" and "Clean Graphics" options, and everyone will be able to understand what that means exactly by looking up the respective pages in the standard.  

Now, looks like we are well equipped to deal with anonymizing our dataset. Let's give this approach a shot and look at a couple of examples. 

One thing to mention: further on in this exercise, assume that we are anonymizing everything only per the Basic Profile, with "Retain UIDs" option, for the sake of simplicity.

Let's find out what to do about Age (0010,1010) and Birth Date (0010,0030) which are clearly PHI per HIPAA:

In [10]:
print(img[(0x0010,0x0030)])
print(img[(0x0010,0x1010)])

(0010, 0030) Patient's Birth Date                DA: '19550101'
(0010, 1010) Patient's Age                       AS: '070Y'


This is what the profile tells us:

<img src="anonymization.img/age_ref.png">

X for Patient's age means "remove", Z for birth date means "replace with a zero length value, or a non-zero length value that may be a dummy value and consistent with the VR".

Let's replace the Birth Date with something that is consistent with the underlying VR (Value Representation). [VR for Birth Date is DA](https://dicom.innolitics.com/ciods/mr-image/patient/00100030) which means "*A string of characters of 
the format YYYYMMDD; where YYYY shall contain year, MM shall contain the month, and DD shall contain the day, interpreted as a date of the Gregorian calendar system.*" (per [VR reference](http://dicom.nema.org/medical/dicom/2020a/output/chtml/part05/sect_6.2.html)). So, proper anonymization of these two per Basic Confidentiality Profile could be:

In [11]:
if "PatientAge" in img:
    del img.PatientAge

In [12]:
img.PatientBirthDate = "19500101"

At this point I will hand it over to you and have you finish the anonymization process for the dataset.

Sometimes you may encounter ambiguities. If you look closer at the list of notes in our Confidentiality profiles page, you fill find this:

> The Attributes listed in Table E.1-1 may not be sufficient to guarantee confidentiality of patient identity. In particular, identifying information may be contained in Private Attributes, new Standard Attributes, Retired Standard Attributes and additional Standard Attributes not present in Standard Composite IODs (as defined in PS3.3) but used in Standard Extended SOP Classes. Table E.1-1 indicates those Attributes that are used in Standard Composite IODs as well as those Attributes that are Retired. Also included in Table E.1-1 are some Elements that are not normally found in a Dataset, but are used in Commands, Directories and Meta Information Headers, but that could be misused within Private Sequences. Textual Content Items of Structured Reports, textual annotations of Presentation States, Curves and Overlays are specifically addressed. It is the responsibility of the de-identifier to ensure that all identifying information is removed.


Basically, what this is saying that it's on you to ensure that the dataset is compliant with respective regulations. So, as you go through the exercise, if there is something in the dataset that seems like PHI, but is not mentioned by the profile table, you should still deal with it taking HIPAA (or other local regulations) into account.

In [20]:
# TASK: write a Python function to de-identify this particular image 
# per the Basic Confidentiality Profile, with "Retain UIDs" option

# <YOUR CODE HERE>

def anonymizeBasic(ds):
    if ("InstanceCreationDate" in ds):
        del ds.InstanceCreationDate
    if ("InstanceCreationTime" in ds):
        del ds.InstanceCreationTime
        
    ds.StudyDate = "19900101"
    ds.SeriesDate = "19900101"
    
    if ("AcquisitionDate" in ds):
        del ds.AcquisitionDate
    
    ds.ContentDate = ""
    ds.StudyTime = ""
    
    if ("SeriesTime" in ds):
        del ds.SeriesTime
    if ("AcquisitionTime" in ds):
        del ds.AcquisitionTime
    if ("ContentTime" in ds):
        del ds.ContentTime
        
    ds.AccessionNumber = "MRKNEE1"
    
    if ("InstitutionName" in ds):
        del ds.InstitutionName
    if ("InstitutionAddress" in ds):
        del ds.InstitutionAddress
    
    ds.ReferringPhysicianName = ""
    
    if ("StudyDescription" in ds):
        del ds.StudyDescription
        
    if ("SeriesDescription" in ds):
        del ds.SeriesDescription
        
    if ("InstitutionalDepartmentName" in ds):
        del ds.InstitutionalDepartmentName
        
    if ("PerformingPhysicianName" in ds):
        del ds.PerformingPhysicianName
        
    if ("OperatorsName" in ds):
        del ds.OperatorsName
        
    ds.PatientName = "Anonymous"
    ds.PatientID = ""
    
    if ("IssuerOfPatientID" in ds):
        del ds.IssuerOfPatientID
        
    ds.PatientBirthDate = "19500101"
    ds.PatientSex = ""
    
    if ("StationName" in ds):
        del ds.StationName
    
    if ("PatientWeight" in ds):
        del ds.PatientWeight
        
    ds.ContrastBolusAgent = ""
    
    if ("DeviceSerialNumber" in ds):
        del ds.DeviceSerialNumber
        
    ds.StudyID = ""
    
    if ("PerformedProcedureStepStartDate" in ds):
        del ds.PerformedProcedureStepStartDate
        
    if ("PerformedProcedureStepStartTime" in ds):
        del ds.PerformedProcedureStepStartTime
        
    if ("PerformedProcedureStepStartID" in ds):
        del ds.PerformedProcedureStepStartID
        
    return ds

## Private DICOM Tags

Now, there is one last thing left. Perhaps, if you've been meticulous, you've noticed tags like the (0019, 1008):

In [15]:
img[(0x0019,0x1008)]

(0019, 1008) [CSA Image Header Type]             CS: 'IMAGE NUM 4'

If you try searching it in the [list of DICOM Data Elements](http://dicom.nema.org/medical/dicom/2020a/output/chtml/part06/chapter_6.html) you won't find it there. If you look carefully at these, you might notice that the group ID is an odd number (0019). Such tags are called "private tags" and are left up to the manufacturer of the system producing DICOMs to store any information they see relevant. They are not defined by the standard. By the way, if you are producing DICOM from your own AI system, it is perfectly legitimate way to store any relevant information (e.g. a model architecture used or maybe a training dataset ID) this way.  



In [16]:
img[(0x0019,0x1008)].tag.is_private

True

If you are obtaining the dataset directly from a clinical archive, you might want to take a closer look at those to see if anything in there might be useful. But our Basic Confidentiality profile has this to say about private data elements:

<img src="anonymization.img/private_attr.png">

In [17]:
# TASK: if you haven't already, extend your function (or write a new one)
# to clean up all Private Attributes per the Basic Confidentiality Profile

# <YOUR CODE HERE>

def removePrivateTags(ds):
    for t in enumerate(ds):
        if (t[1].tag.is_private):
            del ds[t[1].tag]

## Saving anonymized data

And lastly let's see what we need to do to complete our anonymization per the best practices per the Confidentiality Profiles guidelines. Actually, all this is clearly spelled out in [section E.1.1](http://dicom.nema.org/medical/dicom/2020a/output/chtml/part15/chapter_E.html) of our chapter. What's left for us to be able to claim conformance with the profile are steps 6 and 7:

> 6. The attribute Patient Identity Removed (0012,0062) shall be replaced or added to the dataset with a value of YES, and one or more codes from [CID 7050 “De-identification Method”](http://dicom.nema.org/medical/dicom/2020a/output/chtml/part16/sect_CID_7050.html) corresponding to the profile and options used shall be added to De-identification Method Code Sequence (0012,0064). A text string describing the method used may also be inserted in or added to De-identification Method (0012,0063), but is not required. 
> 7. If the Dataset being de-identified is being stored within a DICOM File, then the File Meta Information including the 128 byte preamble, if present, shall be replaced with a description of the de-identifying application. Otherwise, there is a risk that identity information may leak through unmodified File Meta Information or preamble. See PS3.10.

Note that this assumes modifications to the [DICOM Meta Information fields](http://dicom.nema.org/medical/dicom/2020a/output/chtml/part10/chapter_7.html) which includes a 128 byte preample. Note that you can extract the file meta information and preamble using pydicom like so:



In [18]:
print("File Meta: ")
print(img.file_meta)
print("\nPreamble: ")
print(img.preamble)

File Meta: 
(0002, 0000) File Meta Information Group Length  UL: 218
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.2.33.37105.2015011616025459632728241
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.840.113654.2.3.1995.2.12.0
(0002, 0013) Implementation Version Name         SH: 'MIRCTN16NOV2000'
(0002, 0016) Source Application Entity Title     AE: 'DicomBrowser'

Preamble: 
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [19]:
# TASK: Extend your de-id function (or write new code) with the steps above and save the anonymized image series 

# <YOUR CODE HERE>

def anonymizeMeta(ds):
    # Meta information
    
    # Preamble needs to be 128 bytes long
    preamble_str = b"Training Anonymizer"
    preamble = bytearray(preamble_str)
    preamble.extend([0 for _ in range(0,128-len(preamble_str))])

    ds.preamble = preamble
    
    ds.file_meta.ImplementationVersionName = "VERSION_1"
    ds.file_meta.SourceApplicationEntityTitle = "AnonExercise"
    
    # De-id stuff

    ds.PatientIdentityRemoved = "YES"
    ds.DeidentificationMethod = "De-id training exercise" 
    
    ds.DeidentificationMethodCodeSequence = [pydicom.Dataset()]
    ds.DeidentificationMethodCodeSequence[0].CodingSchemeDesignator = "DCM"
    ds.DeidentificationMethodCodeSequence[0].CodeMeaning = "Basic Application Confidentiality Profile"
    ds.DeidentificationMethodCodeSequence[0].CodeValue = "113100"

    ds.DeidentificationMethodCodeSequence += [pydicom.Dataset()]
    ds.DeidentificationMethodCodeSequence[1].CodingSchemeDesignator = "DCM"
    ds.DeidentificationMethodCodeSequence[1].CodeMeaning = "Retain UIDs Option"
    ds.DeidentificationMethodCodeSequence[1].CodeValue = "113110"

In [21]:
def anonymize(ds):
    anonymizeBasic(ds)
    removePrivateTags(ds)
    anonymizeMeta(ds)
    

In [22]:
anonymize(img)

In [23]:
img

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND']
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.2.33.37105.2015011616025459632728241
(0008, 0020) Study Date                          DA: '19900101'
(0008, 0021) Series Date                         DA: '19900101'
(0008, 0023) Content Date                        DA: ''
(0008, 0030) Study Time                          TM: ''
(0008, 0050) Accession Number                    SH: 'MRKNEE1'
(0008, 0060) Modality                            CS: 'MR'
(0008, 0070) Manufacturer                        LO: 'SIEMENS'
(0008, 0090) Referring Physician's Name          PN: ''
(0008, 1090) Manufacturer's Model Name           LO: 'SymphonyTim'
(0008, 1140)  Referenced Image Sequence   3 item(s) ---- 
   (0008, 1150) Referenced SOP Class UID            UI: MR Image 

## Stand out suggestions

If you would like to dig a bit deeper into this, some things you can play with are:

* Try implementing the profile without the "Retain UIDs" option. How would you mangle DICOM UIDs without invalidating DICOM files while maintaining internal consistency?
* Try implementing the ["Retain Longitudinal Temporal Information Option"](http://dicom.nema.org/medical/dicom/2020a/output/chtml/part15/sect_E.3.6.html). This option declares the approach where temporal information about the dataset is retained. For example, you may be building an AI tool that draws inference from a set of images taken from the same patient over some period of time (also called longitudinal studies). In this case, the temporal relationship between such images becomes important and you can still keep the dataset de-identified per HIPAA while retaining this information